<a href="https://colab.research.google.com/github/krjohnn/LU-DM-26m/blob/main/MD1/NOT33doubleNP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import plotly.graph_objects as go

# Make 2 data points
x = torch.tensor([[0.0,1.0]], dtype=torch.double)
y = torch.tensor([1.0,0.0], dtype=torch.double)

w_all = [w for w in range(-15, 10, 1)]
b_all = [b for b in range(-10, 10, 1)]

loss_landscape = []
for b in b_all:
        tmp = []
        for W in w_all:
            b = torch.tensor([b], dtype=torch.double)
            W = torch.tensor([[W]], dtype=torch.double)
            y_pred = torch.sigmoid(W.mm(x) + b)
            loss = (y_pred - y).pow(2).sum()/2
            tmp.append(loss.item())
        loss_landscape.append(tmp)

fig = go.Figure(data=[go.Surface(x=w_all, y=b_all, z=loss_landscape)])

fig.update_layout(scene=dict(xaxis_title='W', yaxis_title='b',zaxis_title='Loss'))

fig.show()


In [ ]:
x_plot = []
y_plot = []
z_plot = []

    # Construct a linear model
b = torch.tensor([-1.0], dtype=torch.double, requires_grad=True)
W = torch.tensor([[4.0]], dtype=torch.double, requires_grad=True)

optimizer = torch.optim.SGD([b,W], lr=50.0)

for t in range(20):
    y_pred = torch.sigmoid(W.mm(x) + b)
    loss = (y_pred - y).pow(2).sum()/2
    print(t, W.item(), b.item(), loss.item())

    x_plot.append(W.item())
    y_plot.append(b.item())
    z_plot.append(loss.item() + 0.03)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

fig.add_trace(go.Scatter3d(x=x_plot, y=y_plot, z=z_plot,
                               mode="lines", line=dict(color="yellow", width=7)))

fig.show()

In [ ]:
x_plot = []
y_plot = []
z_plot = []

    # Construct a linear model
b = torch.tensor([-1.0], dtype=torch.double, requires_grad=True)
W = torch.tensor([[4.0]], dtype=torch.double, requires_grad=True)

optimizer = torch.optim.SGD([b,W], lr=50.0)

for t in range(20):
    y_pred = torch.sigmoid(W.mm(x) + b)
    loss = (y_pred - y).pow(2).sum()/2
    print(t, W.item(), b.item(), loss.item())

    x_plot.append(W.item())
    y_plot.append(b.item())
    z_plot.append(loss.item() + 0.03)

    delta = 0.001
    learning_rate = 50
    y_pred = torch.sigmoid((W+delta).mm(x) + b)
    loss_W = (y_pred - y).pow(2).sum()/2
    y_pred = torch.sigmoid((W).mm(x) + (b+delta))
    loss_b = (y_pred - y).pow(2).sum()/2
    grad_W = (loss_W - loss)/delta
    grad_b = (loss_b - loss)/delta
    b = b - grad_b * learning_rate
    W = W - grad_W * learning_rate

    #optimizer.zero_grad()
    #loss.backward()
    #optimizer.step()

fig.add_trace(go.Scatter3d(x=x_plot, y=y_plot, z=z_plot,
                               mode="lines", line=dict(color="yellow", width=7)))

fig.show()

In [ ]:
import numpy as np

# input and target data
x = np.array([[0], [1]], dtype=np.float64)
y = np.array([[1], [0]], dtype=np.float64)

# Initialize parameters
b = np.array([-1.0], dtype=np.float64)
W = np.array([[4.0]], dtype=np.float64)

# Lists to store plots data
x_plot = []
y_plot = []
z_plot = []

learning_rate = 50.0
delta = 0.001

for t in range(20):
    # Forward pass
    y_pred = 1 / (1 + np.exp(-(np.dot(x, W) + b)))  # Sigmoid function
    loss = np.sum((y_pred - y) ** 2) / 2

    print(t, W, b, loss)

    x_plot.append(W[0,0])
    y_plot.append(b[0])
    z_plot.append(loss + 0.03)

    # Manual gradient calculation
    y_pred_W = 1 / (1 + np.exp(-(np.dot(x, W + delta) + b)))
    loss_W = np.sum((y_pred_W - y) ** 2) / 2
    grad_W = (loss_W - loss) / delta

    y_pred_b = 1 / (1 + np.exp(-(np.dot(x, W) + b + delta)))
    loss_b = np.sum((y_pred_b - y) ** 2) / 2
    grad_b = (loss_b - loss) / delta

    # Parameters update
    b -= learning_rate * grad_b
    W -= learning_rate * grad_W

fig.add_trace(go.Scatter3d(x=x_plot, y=y_plot, z=z_plot,
                               mode="lines", line=dict(color="red", width=7)))

fig.show()


In [ ]:
import numpy as np

# input and target data
x = np.array([[0], [1]], dtype=np.float64)
y = np.array([[1], [0]], dtype=np.float64)

# Initialize parameters
b = np.array([-1.0], dtype=np.float64)
W = np.array([[4.0]], dtype=np.float64)

# Lists to store plots data
x_plot = []
y_plot = []
z_plot = []

learning_rate = 50.0
delta = 0.001

for t in range(20):
    # Forward pass
    y_pred = 1 / (1 + np.exp(-(np.dot(x, W) + b)))  # Sigmoid function
    loss = np.sum((y_pred - y) ** 2) / 2

    print(t, W, b, loss)

    x_plot.append(W[0,0])
    y_plot.append(b[0])
    z_plot.append(loss + 0.03)

    # Derivative of loss with respect to y_pred
    dloss_dy_pred = y_pred - y

    # Derivative of sigmoid
    dy_pred_dz = y_pred * (1 - y_pred)

    # Derivative of loss with respect to z
    dloss_dz = dloss_dy_pred * dy_pred_dz

    # Gradient with respect to W
    grad_W = np.dot(x.T, dloss_dz)

    # Gradient with respect to b
    grad_b = np.sum(dloss_dz)


    # Parameters update
    b -= learning_rate * grad_b
    W -= learning_rate * grad_W

fig.add_trace(go.Scatter3d(x=x_plot, y=y_plot, z=z_plot,
                               mode="lines", line=dict(color="blue", width=7)))

fig.show()